<a href="https://colab.research.google.com/github/richeym-umich/workshops/blob/main/Machine_Learning_Tools_and_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test